In [8]:
import sqlite3

In [9]:
connection = sqlite3.connect("mydb.db")

In [10]:
connection

In [11]:
table_creation_query = """
CREATE TABLE IF NOT EXISTS employees (
emp_id INTEGER PRIMARY KEY,
first_name TEXT NOT NULL,
last_name TEXT NOT NULL,
email TEXT UNIQUE NOT NULL,
hire_date TEXT NOT NULL,
salary REAL NOT NULL
);
"""

In [12]:
table_creation_query2 = """
CREATE TABLE IF NOT EXISTS customers (
customer_id INTEGER PRIMARY KEY,
first_name TEXT NOT NULL,
last_name TEXT NOT NULL,
email TEXT UNIQUE NOT NULL,
phone_number TEXT
);
"""

In [13]:
table_creation_query3 = """
CREATE TABLE IF NOT EXISTS orders (
order_id INTEGER PRIMARY KEY AUTOINCREMENT,
customer_id INTEGER NOT NULL,
order_date TEXT NOT NULL,
amount REAL NOT NULL,
FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
"""

In [14]:
cursor = connection.cursor()

In [15]:
cursor.execute(table_creation_query)
cursor.execute(table_creation_query2)
cursor.execute(table_creation_query3)

In [16]:
#  2. Delete all data from all tables
cursor.execute("DELETE FROM orders;")
cursor.execute("DELETE FROM customers;")
cursor.execute("DELETE FROM employees;")

# 3. Commit the changes
connection.commit()


In [17]:
insert_query = """
INSERT INTO employees (emp_id, first_name, last_name, email, hire_date, salary)
VALUES (?, ?, ?, ?, ?, ?);
"""

insert_query_customers = """
INSERT INTO customers (customer_id, first_name, last_name, email, phone_number)
VALUES (?, ?, ?, ?, ?);
"""

insert_query_orders = """
INSERT INTO orders (order_id, customer_id, order_date, amount)
VALUES (?, ?, ?, ?);
"""

In [18]:
employee_data = [
    (1, "Sunny", "Savita", "sunny.sv@abc.com", "2023-06-01", 50000.00),
    (2, "Arhun", "Meheta", "arhun.m@gmail.com", "2022-04-15", 60000.00),
    (3, "Alice", "Johnson", "alice.johnson@jpg.com", "2021-09-30", 55000.00),
    (4, "Bob", "Brown", "bob.brown@uio.com", "2020-01-20", 45000.00),
    ]

In [19]:
customers_data = [
    (1, "John", "Doe", "john.doe@example.com", "1234567890"),
    (2, "Jane", "Smith", "jane.smith@example.com", "9876543210"),
    (3, "Emily", "Davis", "emily.davis@example.com", "4567891230"),
    (4, "Michael", "Brown", "michael.brown@example.com", "7894561230"),
]


In [20]:
orders_data = [
    (1, 1, "2023-12-01", 250.75),
    (2, 2, "2023-11-20", 150.50),
    (3, 3, "2023-11-25", 300.00),
    (4, 4, "2023-12-02", 450.00),
]

In [21]:
cursor.executemany(insert_query, employee_data)
cursor.executemany(insert_query_customers, customers_data)
cursor.executemany(insert_query_orders, orders_data)

In [22]:
connection.commit()

In [23]:
cursor.execute("SELECT * FROM employees;")

In [24]:
for row in cursor.fetchall():
    print(row)

(1, 'Sunny', 'Savita', 'sunny.sv@abc.com', '2023-06-01', 50000.0)
(2, 'Arhun', 'Meheta', 'arhun.m@gmail.com', '2022-04-15', 60000.0)
(3, 'Alice', 'Johnson', 'alice.johnson@jpg.com', '2021-09-30', 55000.0)
(4, 'Bob', 'Brown', 'bob.brown@uio.com', '2020-01-20', 45000.0)


In [25]:
 from langchain_community.utilities import SQLDatabase

In [26]:
db = SQLDatabase.from_uri("sqlite:///mydb.db")

In [27]:
db

In [28]:
db.dialect

'sqlite'

In [29]:
db.get_usable_table_names()

['customers', 'employees', 'orders']

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.3-70b-versatile", groq_api_key="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")

In [31]:
llm.invoke("Hello, how are you?")

AIMessage(content="Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 41, 'total_tokens': 81, 'completion_time': 0.109137726, 'completion_tokens_details': None, 'prompt_time': 0.007629627, 'prompt_tokens_details': None, 'queue_time': 0.019042071, 'total_time': 0.116767353}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_bebe2dd4fb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd81f-ca81-7ce3-944a-5e9e80a57398-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 41, 'output_tokens': 40, 'total_tokens': 81})

In [32]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [33]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [35]:
tools = toolkit.get_tools()

In [36]:
for tool in tools:
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [37]:
list_tables_tool = next((tool for tool in tools if tool.name == "sql_db_list_tables"), None)


In [38]:
list_tables_tool

ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1321c1ea0>)

In [39]:
get_schema_tool = next((tool for tool in tools if tool.name == "sql_db_schema"), None)


In [40]:
get_schema_tool

InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1321c1ea0>)

In [41]:
print(list_tables_tool.invoke(""))


customers, employees, orders


In [42]:
print(get_schema_tool.invoke("employees"))



CREATE TABLE employees (
	emp_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	hire_date TEXT NOT NULL, 
	salary REAL NOT NULL, 
	PRIMARY KEY (emp_id), 
	UNIQUE (email)
)

/*
3 rows from employees table:
emp_id	first_name	last_name	email	hire_date	salary
1	Sunny	Savita	sunny.sv@abc.com	2023-06-01	50000.0
2	Arhun	Meheta	arhun.m@gmail.com	2022-04-15	60000.0
3	Alice	Johnson	alice.johnson@jpg.com	2021-09-30	55000.0
*/
